In [51]:
''' -------------------- IMPORTS -------------------- '''

import cvxpy as cp            # CVXPy
import numpy as np            # NumPy
import scipy as sp            # SciPy
import superscs               # SuperSCS
import scipy.sparse as sps    # SciPy sparse




''' -------------------- HELPERS -------------------- '''


'''
Helper function quantileLoss
'''
def quantileLoss(alphas, Theta, X, y):
    m, n = X.shape
    k = len(alphas)
    Y = np.tile(y.flatten(), (k, 1)).T
    A = np.tile(alphas, (m, 1))
    Z = X*Theta - Y
    return cp.sum_entries(
        cp.max_elemwise(
            cp.mul_elemwise( -A, Z),
            cp.mul_elemwise(1-A, Z)))

'''
Helper function
'''
def normalized_data_matrix(m, n, mu):
    if mu == 1:
        # dense
        A = np.random.randn(m, n)
        A /= np.sqrt(np.sum(A**2, 0))
    else:
        # sparse
        A = sps.rand(m, n, mu)
        A.data = np.random.randn(A.nnz)
        N = A.copy()
        N.data = N.data**2
        A = A*sps.diags([1 / np.sqrt(np.ravel(N.sum(axis=0)))], [0])
    return A
    

''' -------------------- BENCHMARKS -------------------- '''

'''
Basis Pursuit problems
problemOptions = {'n':500, 'm':300, 'density':0.1}
'''
def basisPursuitProblem(problemOptions, solverOptions):
    A = np.random.rand(problemOptions['m'], problemOptions['n'])
    x0 = sps.rand(problemOptions['n'], 1, 0.1)
    b = A*x0
    x = cp.Variable( problemOptions['n'])
    prob = cp.Problem(cp.Minimize(cp.norm1(x)), [A*x == b])
    prob.solve(**solverOptions)
    return {'Problem':prob, 'name':'basisPursuitProblem'}

'''
Chebyshev
problemOptions = {'n':200, 'm':100, 'k':10}
'''    
def chebyshevProblem(problemOptions, solverOptions):
    n = problemOptions['n']
    m = problemOptions['m']
    k = problemOptions['k']
    A = [normalized_data_matrix(m,n,1) for i in range(k)]
    B = normalized_data_matrix(k,n,1)
    c = np.random.rand(k)
    # Problem construction
    x = cp.Variable(n)
    obj_list = [cp.pnorm(A[i]*x, 2) + cp.abs(B[i,:]*x - c[i]) for i in range(k)]
    f = cp.max_elemwise(obj_list)
    prob = cp.Problem(cp.Minimize(f))
    return {'Problem':prob, 'name':'chebyshevProblem'}
    

    
'''
Chebychev epigraph
problemOptions = {'n':200, 'm':100, 'k':10}
'''    
def chebyshevEpigraphProblem(problemOptions, solverOptions):
    n = problemOptions['n']
    m = problemOptions['m']
    k = problemOptions['k']
    A = [normalized_data_matrix(m,n,1) for i in range(k)]
    B = normalized_data_matrix(k,n,1)
    c = np.random.rand(k)
    x = cp.Variable(n)
    t = cp.Variable(k)
    f = cp.max_entries(t+cp.abs(B*x-c))
    C = []
    for i in range(k):
        C.append(cp.pnorm(A[i]*x, 2) <= t[i])
    prob = cp.Problem(cp.Minimize(f), C)
    return {'Problem':prob, 'name':'chebyshevEpigraphProblem'}

'''
Quantile
problemOptions = {'n':10, 'm':400, 'k':100, 'p':1, 'sigma':0.1}
'''    
def quantileProblem(problemOptions, solverOptions):
    n = problemOptions['n']
    m = problemOptions['m']
    k = problemOptions['k']
    p = problemOptions['p']
    sigma = problemOptions['sigma']
    # Variable declarations
    Theta = cp.Variable(n,k)
    x = np.random.rand(m)*2*np.pi*p
    y = np.sin(x) + sigma*np.sin(x)*np.random.randn(m)
    alphas = np.linspace(1./(k+1), 1-1./(k+1), k)
    # RBF features
    mu_rbf = np.array([np.linspace(-1, 2*np.pi*p+1, n)])
    mu_sig = (2*np.pi*p+2)/n
    X = np.exp(-(mu_rbf.T - x).T**2/(2*mu_sig**2))
    f = quantileLoss(alphas, Theta, X, y)
    C = [X*(Theta[:,1:] - Theta[:,:-1]) >= 0]
    prob = cp.Problem(cp.Minimize(f), C)
    prob.solve(**solverOptions)
    return {'Problem':prob, 'name':'quantileProblem'}

'''
Control
problemOptions = {'m':2, 'n':8, 'T':50, 'alpha':0.2, 'beta':5}
'''
def controlProblem(problemOptions, solverOptions):
    m = problemOptions['m'] # number of inputs
    n = problemOptions['n'] # number of states
    T = problemOptions['T'] # number of time steps
    alpha = problemOptions['alpha']
    beta = problemOptions['beta']
    A = np.eye(n) + alpha*np.random.randn(n,n)
    B = np.random.randn(n,m)
    x_0 = beta*np.random.randn(n,1)
    
    # Problem construction
    # Form and solve control problem.
    x = cp.Variable(n, T+1)
    u = cp.Variable(m, T)
    states = []
    
    for t in range(T):
        cost = cp.pnorm(u[:,t], 1)
        constr = [x[:,t+1] == A*x[:,t] + B*u[:,t],
                  cp.norm(u[:,t], 'inf') <= 1]
        states.append(cp.Problem(cp.Minimize(cost), constr))
        
    # sums problem objectives and concatenates constraints.
    prob = sum(states)
    prob.constraints += [x[:,T] == 0, x[:,0] == x_0]
    return {'Problem':prob, 'name':'controlProblem'}


'''
Covsel
problemOptions = {'m':20, 'n':30, 'lam':0.01, 'density': 0.01, 'beta':0.1}
'''
def covselProblem():
    m = problemOptions['m']
    n = problemOptions['n']
    lam = problemOptions['lam']
    A = sps.rand(n,n, problemOptions['density'])
    A = np.asarray(A.T.dot(A).todense() + problemOptions['beta']*np.eye(n))
    L = np.linalg.cholesky(np.linalg.inv(A))
    X = np.random.randn(m,n).dot(L.T)
    S = X.T.dot(X)/m
    W = np.ones((n,n)) - np.eye(n)
    # Problem construction
    Theta = cp.Variable(n,n)
    prob = cp.Problem(cp.Minimize(
            lam*cp.norm1(cp.mul_elemwise(W,Theta)) +
            cp.sum_entries(cp.mul_elemwise(S,Theta)) -
            cp.log_det(Theta)))
    return {'Problem':prob, 'name':'covselProblem'}



In [60]:
# Example:
np.random.seed(0) # for reproducibility
solverOptions = {'solver': 'SUPERSCS','memory': 100, 'eps': 1e-3}
solverOptionsSCS = {'solver': 'SCS'}
problemOptions = {'n':1000, 'm':500, 'density':0.1} # problem options for basis pursuit
P = basisPursuitProblem(problemOptions, solverOptions)
print(P['Problem'].solver_stats.solve_time)

1.588524274
